In [13]:
import matplotlib.pyplot as plt
import numpy.random
import numpy as np
import seaborn
%pylab inline
pylab.rcParams['figure.figsize'] = (8, 8)
#seaborn.set_context("talk")

Populating the interactive namespace from numpy and matplotlib


In [3]:
import sklearn.naive_bayes #MultinomialNB
from collections import Counter

In [18]:
!head -n 3 rt-polarity.neg

simplistic , silly and tedious . 
it's so laddish and juvenile , only teenage boys could possibly find it funny . 
exploitative and largely devoid of the depth or sophistication that would make watching such a graphic treatment of the crimes bearable . 


In [19]:
!head -n 3 rt-polarity.pos

the rock is destined to be the 21st century's new " conan " and that he's going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal . 
the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson's expanded vision of j . r . r . tolkien's middle-earth . 
effective but too-tepid biopic


In [22]:
data_neg = [l.strip().split() for l in open("rt-polarity.neg", encoding = "ISO-8859-1")]
data_pos = [l.strip().split() for l in open("rt-polarity.pos", encoding = "ISO-8859-1")]

dictionary = {}
rev_dictionary = {}
counts = Counter()
for sent in data_pos + data_neg:
    for word in sent:
        counts[word] += 1
        if word not in dictionary:
            key = len(dictionary)
            dictionary[word] = key
            rev_dictionary[key] = word


In [24]:
npos = len(data_pos)
nneg = len(data_neg)
n = npos + nneg
m = len(dictionary)
X = np.zeros((n, m))
y = np.zeros((n,))
y[:npos] = 1

In [25]:
for i,sent in enumerate(data_pos + data_neg):
    for word in sent:
        j = dictionary[word]
        X[i, j] += 1

In [26]:
def encode(sent):
    x = np.zeros((1, m))
    for word in sent.split():
        if word in dictionary:
            x[0, dictionary[word]] += 1
    return x

In [31]:
model = sklearn.naive_bayes.MultinomialNB()

In [32]:
X.shape, y.shape

((10662, 21401), (10662,))

In [33]:
model.fit(X, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
features = []
for word, c in counts.most_common(1000):
    j = dictionary[word]
    pi = model.feature_log_prob_[:, j]
    if c > 500:
        print("%10s \t %d \t %s"%(word, j, pi))
    features.append((word, pi[1]/pi[0]))

         . 	 30 	 [-2.93901082 -2.95454721]
       the 	 0 	 [-3.27148481 -3.27680855]
         , 	 23 	 [-3.32813063 -3.23475216]
         a 	 16 	 [-3.65123352 -3.55346689]
       and 	 11 	 [-3.9166723  -3.62964515]
        of 	 34 	 [-3.87851246 -3.69873863]
        to 	 4 	 [-4.07290218 -4.21855921]
        is 	 2 	 [-4.38305711 -4.36068435]
        in 	 122 	 [-4.63505983 -4.60392314]
      that 	 12 	 [-4.70068337 -4.66310291]
        it 	 81 	 [-4.72242335 -4.8003739 ]
        as 	 71 	 [-4.97060298 -5.02442613]
       but 	 55 	 [-5.04596184 -5.14121534]
      with 	 118 	 [-5.27908493 -5.01876039]
      film 	 85 	 [-5.36208185 -5.09139183]
      this 	 146 	 [-5.14536919 -5.30155799]
       for 	 252 	 [-5.15966443 -5.29111792]
       its 	 158 	 [-5.33989394 -5.2532677 ]
        an 	 74 	 [-5.4561108  -5.17498478]
     movie 	 76 	 [-5.16491379 -5.56797844]
      it's 	 129 	 [-5.47387326 -5.4675223 ]
        be 	 5 	 [-5.54248218 -5.76409332]
        on 	 416 	 [-5.6412532

In [35]:
features.sort(key=lambda x: x[1])
for w, s in features[:10]:
    print("%10s %f"%( w, s))

engrossing 0.748111
 wonderful 0.776461
      warm 0.776461
 inventive 0.776788
  powerful 0.778615
refreshing 0.784623
  captures 0.788538
  provides 0.788538
  touching 0.791721
  portrait 0.799052


In [36]:
for w, s in features[-10:]:
    print("%10s %f"%( w, s))


    stupid 1.269816
   routine 1.295109
  mediocre 1.301057
   generic 1.301057
       bad 1.310359
      dull 1.315481
      flat 1.316063
    boring 1.337208
     badly 1.369386
   unfunny 1.388356


In [37]:
model.predict(encode("the film was excellent"))

array([ 1.])

In [38]:
model.predict(encode("professor parkes lecture was an engrossing portait of Bayesian ideals"))

array([ 1.])

In [122]:
model.predict_proba(encode("professor parkes lecture was an engrossing portait of Bayesian ideals"))

array([[ 0.25657334,  0.74342666]])

In [39]:
model.predict_proba(encode("professor rush boldly satirized the boring flat dull routine of modern existence"))

array([[  9.99937145e-01,   6.28550516e-05]])

In [24]:
import sklearn.linear_model

In [25]:
model_lr = sklearn.linear_model.LogisticRegression()

In [26]:
model_lr.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)